In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

In [1]:
# Load Packages
import tensorflow as tf
from tensorflow.keras import backend
#from __future__ import print_function
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop, Adam
from keras.utils.data_utils import get_file
from keras.initializers import glorot_uniform
from keras.models import load_model, save_model
import keras
import numpy as np
import pandas as pd
import random
import sys
import io
import re
import os

Using TensorFlow backend.


In [ ]:
os.chdir('/content/drive/My Drive/Colab Notebooks/NLP Group Project/')

In [2]:
# Read Songs
songs = pd.read_csv('drake-songs.csv')

In [3]:
text = ''

for index, row in songs['lyrics'].iteritems():
    cleaned = str(row).lower().replace(' ', '\n')
    text = text + " ".join(re.findall(r"[a-z']+", cleaned))
    
len(text)

367372

In [4]:
tokens = re.findall(r"[a-z'\s]", text)

chars = sorted(list(set(tokens)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 28


In [5]:
maxlen = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    
print('nb sequences:', len(sentences))

nb sequences: 122444


In [6]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [7]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [8]:
model = tf.keras.models.load_model('drake_character_model.h5')

In [9]:
def generate_text(model, seed, length, diversity):

    maxlen = 40
    generated = ''
    sentence = text[seed: seed + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(length):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [10]:
import warnings
warnings.filterwarnings('ignore')
generate_text(model, 1234, 200, 0.5)

----- Generating with seed: "nt things to go my way but as of late a "
nt things to go my way but as of late a to take there i got a grave you the crib yours that works that i was you can't stay that do an my tice be yionribs with that i is salzition get it i want to my face you in the beathat is to blies i ca


Predicted!